<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 对象跟随- 基础版

在这个历程中，我们将展示如何使用JetBot跟踪对象!
我们将使用一个在[COCO dataset](http://cocodataset.org)上训练过的预训练神经网络来检测90个不同的常见对象。这些包括

* Person(人) (index 0)
* Cup(杯子) (index 47)

不管怎样，我们开始吧。首先，我们先导入``ObjectDetector``类，它使用我们预先训练的SSD引擎。还有许多其他类(你可以检查[这个文件](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbt)以获得类索引的完整列表)。该模型来自[TensorFlow对象检测API](https://github.com/tensorflow/models/tree/master/research/object_detection)，该API还为定制任务的对象检测器培训提供实用程序!一旦模型被训练，我们使用NVIDIA TensorRT对Jetson Nano进行优化。这使得网络非常快，能够实时控制Jetbot!但是，我们不会在这个笔记本中运行所有的培训和优化步骤。不管怎样，我们开始吧。首先，我们要导入“ObjectDetector”类，该类使用我们预先训练的SSD引擎。

### 单张相机图像检测

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

在内部，' ' ObjectDetector ' '类使用TensorRT Python API来执行我们提供的引擎。它还负责对神经网络的输入进行预处理，以及对检测到的对象进行解析。目前，它只适用于使用``jetbotssd_tensorrt``包创建的引擎。该包具有将模型从TensorFlow对象检测API转换为优化的TensorRT引擎的实用程序。
接下来，让我们初始化相机。我们的检测器需要300x300像素的输入，所以我们会在创建相机时设置这个。

>在内部，Camera类使用GStreamer来利用Jetson Nano的图像信号处理器(ISP),获取图像的速度是非常可观的.

>并且还从CPU卸载了大量的大小计算。

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)
#camera = Camera.instance(width=220, height=220, fps=5)

现在，让我们使用一些摄像机输入来执行我们的网络。在默认情况下，``ObjectDetector``类期望相机生成的格式为``bgr8``。
然而，如果输入格式不同，可以覆盖默认的预处理函数。

如果相机的视场中有任何COCO对象，它们现在应该存储在``detections``变量中。
### 在文本区域显示检测的对象

我们将使用下面的代码打印出检测到的对象。

In [4]:
detections = model(camera.value)

print(detections)

[[{'label': 62, 'confidence': 0.724090576171875, 'bbox': [0.5345957279205322, 0.2678258419036865, 0.9672387838363647, 0.7407461404800415]}, {'label': 62, 'confidence': 0.47122326493263245, 'bbox': [0.049342721700668335, 0.2999565601348877, 0.25558164715766907, 0.704258918762207]}, {'label': 84, 'confidence': 0.37387582659721375, 'bbox': [0.28033605217933655, 0.3279327154159546, 0.32179489731788635, 0.42697060108184814]}, {'label': 84, 'confidence': 0.3250918388366699, 'bbox': [0.3426630198955536, 0.3813009560108185, 0.4206192195415497, 0.41232797503471375]}]]


In [17]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value="[[{'label': 62, 'confidence': 0.8436375856399536, 'bbox': [0.1384977400302887, 0.2862080037593…

您应该看到每个图像中检测到的每个对象的标签、置信度和边框位置。在这个例子中只有一个图像(我们的相机)。

要打印第一张图像中检测到的第一个对象，我们可以调用以下命令

>如果没有检测到对象，这可能会抛出一个错误

In [18]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 62, 'confidence': 0.8436375856399536, 'bbox': [0.1384977400302887, 0.28620800375938416, 0.8189680576324463, 0.963363766670227]}


> 如果没有检测到对象，这里可能会抛出一个错误

# 控制机器人跟随中心物体

现在，我们希望机器人跟随指定类的对象。为此，我们将做以下工作

1.  检测与指定类匹配的对象
2.  选择距离相机视野中心最近的物体，这是目标物体
3.  引导机器人向目标物体移动，否则会产生漂移

我们还将创建一些小部件，用于控制目标对象标签、机器人速度和转弯增益，根据目标对象和机器人视野中心之间的距离来控制机器人转弯的速度。

In [8]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
# ------------------------
# collision_model = torchvision.models.alexnet(pretrained=False)
# collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
# collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
# device = torch.device('cuda')
# collision_model = collision_model.to(device)
# ---------------------------
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #图片缩放至224,224对比224,244的避障模型
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

创建驱动电机的robot实例.

In [9]:
from jetbot import Robot

robot = Robot()

最后，让我们显示所有控件小部件，并将网络执行功能连接到相机更新。

In [5]:
from jetbot import Robot
robot = Robot()
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
import time
# ---------------------------
# blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
# ----------------------------
image_widget = widgets.Image(format='jpeg', width=300, height=300)
#将下面这条语句的value值改为检测目标物体的对象值，取值范围0-99，例如跟随对象是Person(人) (index 1)，需要将value的值改为1
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.9, min=0.0, max=2.0, description='turn gain')
# -----------------------
# display(widgets.VBox([
#     widgets.HBox([image_widget, blocked_widget]),
#     label_widget,
#     speed_widget,
#     turn_gain_widget
# ]))
# -----------------------------
display(widgets.VBox([
    widgets.HBox([image_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """计算对象的中心x、y坐标"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """计算二维向量的长度"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """查找最接近图像中心的检测"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    # 用来判断当机器人视野中没有跟踪目标时，机器人该往那个方向转向
    global lastcenter 
    image = change['new']
    
    # ------------------------
     # 执行冲突模型以确定是否阻塞
#     collision_output = collision_model(preprocess(image)).detach().cpu()
#     prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
#     blocked_widget.value = prob_blocked
    
#     # 如果受阻则左转
#     if prob_blocked >0.95:
#         robot.backward(0.7)
#         image_widget.value = bgr8_to_jpeg(image)
#          # 更新图像显示至小部件
#         image_widget.value = bgr8_to_jpeg(image)
#         return
    
    # ---------------------------
    # 计算所有检测到的对象
    detections = model(image)
    
    # 在图像上绘制所有检测
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # 选择匹配所选类标签的检测
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # 让检测最接近视野中心，并绘制它
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # 如果没有检测到目标，则去寻找目标
    if det is None:
        pass
        # 第一次启动目标跟随功能，让jetbot原地向左转动寻找目标
        if lastcenter == "":
            robot.left(0.7)
        else:
           # 当视野中无目标时，让机器人朝之前目标运动的方向上去寻找目标
            if lastcenter[0] < 0:
                robot.left(0.7)
            else:
                robot.right(0.7)
       
    # 有的话就控制Jetbot去跟随设定的对象
    else:
        # 将机器人向前移动，并控制成比例的目标与中心的x距离
        center = detection_center(det)
        lastcenter = center
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # 更新图像显示至小部件
    image_widget.value = bgr8_to_jpeg(image)
    
lastcenter = ""    
# execute({'new': camera.value})

调用下面的块将执行函数连接到每个摄像机帧更新。
> ## 注意注意:以下函数由于之前通过画面变化来调用检测函数延迟比较大， 现在暂时采用循环不断调用检测函数提高画面流畅度，以及提高识别效果，由于在主线程中不断循环检测会导致卡死在这里并且使得jupyterlab的所有按键失灵，所以采用将不断检测的死循环放到一个子线程1里面

In [6]:
#camera.unobserve_all()
#camera.observe(execute, names='value')

import threading
import inspect
import ctypes
'''以下为定义关闭线程函数'''
def _async_raise(tid, exctype):
  """raises the exception, performs cleanup if needed"""
  tid = ctypes.c_long(tid)
  if not inspect.isclass(exctype):
    exctype = type(exctype)
  res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
  if res == 0:
    raise ValueError("invalid thread id")
  elif res != 1:
    # """if it returns a number greater than one, you're in trouble,
    # and you should call it again with exc=NULL to revert the effect"""
    ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
    raise SystemError("PyThreadState_SetAsyncExc failed")
def stop_thread(thread):
  _async_raise(thread.ident, SystemExit)
'''线程1的函数，里面不断调用检测函数'''
def test():
    while True:
        execute({'new': camera.value})        

        
thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()

+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++


如果机器人没有被阻挡，你应该可以看到蓝色的方框围绕着被探测到的物体,目标对象(机器人跟随的对象)将显示为绿色。
当目标被发现时，机器人应该转向目标。
你可以调用下面的代码块来手动断开与摄像机的处理并停止机器人。

In [7]:
import time
stop_thread(thread1)
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

+++++++
+++++++
+++++++
+++++++
